In [11]:
%pip install numpy
%pip install opencv-python
import numpy as np
import cv2


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Camshift simples, identificando retrovisor de carro. (Exemplo adaptado do código fornecido pela documentação do OpenCV)

In [17]:
cap = cv2.VideoCapture("slow_traffic_small.mp4")

if not cap.isOpened():
    print("Erro ao abrir o vídeo.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('camshift-output.avi', fourcc, fps, (width, height))

ret, frame = cap.read()

x, y, w, h = 300, 200, 100, 50
track_window = (x, y, w, h)

roi = frame[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

    ret, track_window = cv2.CamShift(dst, track_window, term_crit)

    pts = cv2.boxPoints(ret)
    pts = np.int0(pts)
    img2 = cv2.polylines(frame, [pts], True, 255, 2)

    out.write(img2)
    cv2.imshow('CAMShift Tracking', img2)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

C:\Users\hugor\AppData\Local\Temp\ipykernel_7708\3989740469.py:47: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(pts)


Extended Camshift Tracker, criado apartir do estudo de David Exner (doi: 10.1109/CVPRW.2010.5543787) usando o exemplo anterior da OpenCV como base

In [24]:
class ExtendedCAMShiftTracker:
    def __init__(self):
        self.accumulated_hist = None
        self.track_window = None
        self.term_crit = None

    def initialize_tracker(self, frame, x, y, w, h):
        self.track_window = (x, y, w, h)

        roi = frame[y:y+h, x:x+w]
        hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

        self.accumulated_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
        cv2.normalize(self.accumulated_hist, self.accumulated_hist, 0, 255, cv2.NORM_MINMAX)

        self.term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

    def track_frame(self, frame, out):

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        dst = cv2.calcBackProject([hsv], [0], self.accumulated_hist, [0, 180], 1)

        ret, self.track_window = cv2.CamShift(dst, self.track_window, self.term_crit)

        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, 255, 2)

        out.write(img2)
        cv2.imshow('Extended CAMShift Tracking', img2)

        return frame

    def accumulate_histograms(self, reference_histograms):
        self.accumulated_hist = np.sum(reference_histograms, axis=0)
        self.accumulated_hist = cv2.normalize(self.accumulated_hist, self.accumulated_hist, 0, 255, cv2.NORM_MINMAX)

class HierarchicalRedetection:
    def __init__(self, width, height):
        self.root_search_window = (0, 0, width, height)
        self.threshold = 0.8  # Definido arbitrariamente

    def check_exit_condition(self, frame):
        return np.sum(frame) < self.threshold

    def divide_into_quadrants(self, region):
        x, y, w, h = region
        half_width = w // 2
        half_height = h // 2

        quadrant1 = (x, y, half_width, half_height)
        quadrant2 = (x + half_width, y, half_width, half_height)
        quadrant3 = (x, y + half_height, half_width, half_height)
        quadrant4 = (x + half_width, y + half_height, half_width, half_height)

        return [quadrant1, quadrant2, quadrant3, quadrant4]

    def perform_redetection(self, frame, tracker):
        quadrants = self.divide_into_quadrants(self.root_search_window)
        for quadrant in quadrants:
            subframe = frame[quadrant[1]:quadrant[1]+quadrant[3], quadrant[0]:quadrant[0]+quadrant[2]]
            if self.check_exit_condition(subframe):
                x, y, w, h = quadrant
                tracker.initialize_tracker(frame, x, y, w, h)

def main():
    cap = cv2.VideoCapture("slow_traffic_small.mp4")

    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        exit()

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('extended_camshift_output.avi', fourcc, fps, (width, height))

    ret, frame = cap.read()

    tracker = ExtendedCAMShiftTracker()
    tracker.initialize_tracker(frame, 300, 200, 100, 50)

    redetection = HierarchicalRedetection(width, height)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        tracker.initialize_tracker(frame, 300, 200, 100, 50)

        frame = tracker.track_frame(frame, out)

        redetection.perform_redetection(frame, tracker)

        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


C:\Users\hugor\AppData\Local\Temp\ipykernel_7708\475527574.py:28: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(pts)
